In [1]:
import pandas as pd
import numpy as np
import os
import random
import matplotlib.pyplot as plt
from datetime import datetime
import re
import joblib
import json
import math
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve

In [2]:
# fix the value_df
"""
df_header = pd.read_csv("post_data/final_train_Ft.csv", nrows=1)
col_list = df_header.columns.to_list()[:]
col_list = col_list[1:]
print(col_list)
value_df = pd.read_csv("post_data/value_Ft.csv")
for e in col_list:
    if e not in value_df.columns:
        value_df[e] = pd.Series( np.zeros(shape=(len(value_df),)) )
        value_df[e].astype(np.uint8)

value_df = value_df[col_list]
value_df.to_csv("post_data/final_value_Ft.csv", index=False)
"""

'\ndf_header = pd.read_csv("post_data/final_train_Ft.csv", nrows=1)\ncol_list = df_header.columns.to_list()[:]\ncol_list = col_list[1:]\nprint(col_list)\nvalue_df = pd.read_csv("post_data/value_Ft.csv")\nfor e in col_list:\n    if e not in value_df.columns:\n        value_df[e] = pd.Series( np.zeros(shape=(len(value_df),)) )\n        value_df[e].astype(np.uint8)\n\nvalue_df = value_df[col_list]\nvalue_df.to_csv("post_data/final_value_Ft.csv", index=False)\n'

In [3]:
final_value_df = pd.read_csv("post_data/final_value_Ft.csv")
final_value_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4577464 entries, 0 to 4577463
Data columns (total 83 columns):
 #   Column                Dtype  
---  ------                -----  
 0   C1_1001               int64  
 1   C1_1002               int64  
 2   C1_1005               int64  
 3   C1_1007               int64  
 4   C1_1008               int64  
 5   C1_1010               int64  
 6   C1_1012               int64  
 7   banner_pos_0          int64  
 8   banner_pos_1          int64  
 9   banner_pos_2          int64  
 10  banner_pos_3          int64  
 11  banner_pos_4          int64  
 12  banner_pos_5          float64
 13  banner_pos_7          int64  
 14  device_type_0         int64  
 15  device_type_1         int64  
 16  device_type_2         float64
 17  device_type_4         int64  
 18  device_type_5         int64  
 19  device_conn_type_0    int64  
 20  device_conn_type_2    int64  
 21  device_conn_type_3    int64  
 22  device_conn_type_5    int64  
 23  C15_120

In [4]:
#model_bdt_1m = joblib.load( "Model/final_BDT_training_1m_Nblc" )
test_id = pd.read_csv("data/test.csv", usecols=['id'], dtype=str)
test_id['id'] = test_id['id'].astype("str")
#predictions = model_bdt_1m.predict_proba( final_value_df )

#output = pd.DataFrame({'id':test_id['id'], 'click': predictions[:,1]})
#output.to_csv('submission_BDT.csv', index=False)

In [5]:
# models ensembling
def Training_EnsembleModel_predict(X):
    from sklearn.ensemble import VotingClassifier
    model_lr_2m = joblib.load( "Model/final_LR_training_2m_Nblc" )
    model_rf_1m = joblib.load( "Model/final_RF_training_1m_Nblc" )
    model_bdt_1m = joblib.load( "Model/final_BDT_training_1m_Nblc" )
    model = VotingClassifier(estimators=[
            ('lr', model_lr_2m), ('rf', model_rf_1m), ('bdt', model_bdt_1m)],
            voting='soft')
    return model.predict(X)

In [6]:
model_esm_1 = joblib.load( "Model/final_Ensemble_training_1m_esm" )
predictions_esm1 = model_esm_1.predict_proba( final_value_df )
output = pd.DataFrame({'id':test_id['id'], 'click': predictions_esm1[:,1]})
output.to_csv('submission_ESM1.csv', index=False)